# Imports

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn  as sns
import pymysql 

# Getting Data and some cleaning

In [3]:
conn=pymysql.connect(host='localhost',user='root',password='12345678',database='trening')

In [4]:
df=pd.read_sql('select * from c180308',con=conn)

C:\Users\u\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
df

,id,date,avtosalon,salon_yn,marka,model,shekil_sayi,satilib,baxis_sayi,elan_tarix_title,...,yagish,qapanma,radar,kondisioner,podoqrev,deri,ksenon,kamera,perde,ventilyasiya
0,1,2018-03-08 22:12:54,none,0,Mercedes,C 200,9,yox,107,Yeniləndi,...,0,1,0,1,0,0,1,0,0,0
1,2,2018-03-08 22:12:54,none,0,Mitsubishi,Pajero,5,yox,78,Yeniləndi,...,0,1,1,1,0,0,0,1,0,0
2,3,2018-03-08 22:12:54,none,0,Mercedes,C 220,7,yox,118,Yeniləndi,...,1,1,0,1,0,0,0,0,0,0
3,4,2018-03-08 22:12:54,none,0,Volkswagen,Touareg,10,yox,1639,Yeniləndi,...,1,1,1,1,1,1,1,1,1,1
4,5,2018-03-08 22:12:54,none,0,Hyundai,Sonata,8,yox,227,Yeniləndi,...,1,1,1,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25663,25664,2018-03-08 23:29:24,none,0,BMW,320,9,yox,988,Yeniləndi,...,0,1,0,1,0,0,1,0,1,1
25664,25665,2018-03-08 23:29:24,"Avtosalon ""Gənclik Satış Mərkəzi""",1,Baic,X 55,9,yox,10782,Yeniləndi,...,1,1,1,1,0,1,0,0,0,0
25665,25666,2018-03-08 23:29:24,none,0,GAZ,M-21,9,yox,3848,Yeniləndi,...,0,0,0,0,0,0,0,0,0,0
25666,25667,2018-03-08 23:29:24,"Avtosalon ""Gənclik Satış Mərkəzi""",1,Baic,X 55,9,yox,10029,Yeniləndi,...,0,1,1,1,0,1,0,0,0,0


In [7]:
df['muherrik_guc2']=df['muherrik_guc'].str.replace(' a.g.','').astype('float64')

C:\Users\u\AppData\Local\Temp\ipykernel_12468\802459562.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['muherrik_guc2']=df['muherrik_guc'].str.replace(' a.g.','').astype('float64')


In [26]:
df['yurush2']=df['yurush'].str.replace(' km','')

In [28]:
df['yurush2'].dtype

dtype('O')

In [29]:
df['price']=np.select(condlist=[df.qiymet_valyuta=='$',df.qiymet_valyuta=='€'],
                      choicelist=[df.qiymet*1.7,df.qiymet*2.09],default=df.qiymet)

# Basic Statistics Functions

In [31]:
sample=df.sample(20,random_state=1)

In [102]:
sample1=df[(df['marka']=='BMW')&(df['price']>10000)].sample(21,random_state=1)
sample2=df[(df['marka']=='BMW')&(df['price']<10000)].sample(21,random_state=1)

In [123]:
sample3=df[(df['marka']=='BMW')&(df['price']>10000)].sample(21,random_state=3)
sample4=df[(df['marka']=='BMW')&(df['price']<10000)].sample(21,random_state=4)
sample5=df[(df['marka']=='Mercedes')&(df['price']>1000)].sample(21,random_state=30)


In [33]:
sample['price'].mean(), df['price'].mean()

(17104.6, 18461.090717235467)

In [34]:
sample['price'].var(), df['price'].var()

(769747781.0947366, 901332711.5738552)

# Tests

# 1. One sample tests

## 1.1 Parametric tests

## 1.1.1. t-test
test for the null hypothesis that the expected value (mean) of a sample of independent observations 
a is equal to the given population mean, popmean.

In [47]:
ss.ttest_1samp(sample['price'],30000)

Ttest_1sampResult(statistic=-2.07862235161242, pvalue=0.05144276897271192)

## 1.1.2. Chi-square test for variance/standard deviation
tests whether the population variance or standard deviation is equal to a specified value.

In [37]:
sample['shekil_sayi'].var()

13.85263157894737

In [39]:
sample['shekil_sayi'].var(ddof=2)

14.622222222222225

In [46]:
ss.chisquare(sample['shekil_sayi'],ddof=1)

Power_divergenceResult(statistic=28.60869565217392, pvalue=0.053376117720560036)

## 1.2. Non-parametric tests

## 1.2.1. Wilcoxon signed-rank test
tests a hypothesized median, non-parametric alternative of one sample t-test

In [67]:
ss.wilcoxon(sample['price']-10000)

WilcoxonResult(statistic=90.0, pvalue=0.5958194732666016)

In [66]:
sample['price'].median()

8950.0

## 1.2.2. Kolmogorov-Smirnov test
compares the underlying distribution F(x) of a sample against a given continuous distribution G(x).

In [91]:
x=sample['shekil_sayi']

In [95]:
ss.ks_1samp(sample['muherrik_guc2'], ss.norm.cdf,alternative='greater')

KstestResult(statistic=0.0, pvalue=1.0)

In [94]:
ss.ks_1samp(x, ss.norm.cdf,alternative='less')

KstestResult(statistic=0.9999997133484281, pvalue=1.4030933088023867e-131)

# 2. Two sample tests

## 2.1. Parametric tests

## 2.1.1 F-test
tests the null hypothesis that 2 samples are from populations with equal variances.

In [117]:
F=sample1['price'].var(ddof=1)/sample2['price'].var(ddof=1)

In [111]:
ss.f.cdf(F,20,20)

0.9999999999999999

In [124]:
F1=sample1['shekil_sayi'].var(ddof=1)/sample5['shekil_sayi'].var(ddof=1)

In [125]:
ss.f.cdf(F1,20,20)

0.45838794268943533

## 2.1.2. Pooled variance t-test
test for the null hypothesis that 2 independent samples have identical average (expected) values, 
assumes that the populations have identical variances by default.

In [126]:
ss.ttest_ind(sample2['price'],sample5['price'])

Ttest_indResult(statistic=-2.1334973780971986, pvalue=0.039068833081602146)

## 2.1.3. Separate variance t-test
same as pooled variance t-test, but does not assume equal variances

## 2.1.4. Paired samples t-test
test for the null hypothesis that two related or repeated samples have identical average (expected) values.

In [133]:
ss.ttest_rel(sample2['shekil_sayi'],sample4['shekil_sayi'])

Ttest_relResult(statistic=-0.6285393610547089, pvalue=0.5367634575209301)

## 2.2. Non-parametric tests

## 2.2.1. Wilcoxon signed-rank test
tests the null hypothesis that two related paired samples come from the same distribution. 
In particular, it tests whether the distribution of the differences x - y is symmetric about zero. 
It is a non-parametric version of the paired T-test.

In [135]:
ss.wilcoxon(sample2['shekil_sayi'],sample4['shekil_sayi'])

WilcoxonResult(statistic=86.0, pvalue=0.7159770457605454)

## 2.2.2. Mann-Whitney U rank test
tests the distribution underlying sample x is the same as the distribution underlying sample y. 

In [139]:
ss.mannwhitneyu(sample2['price'],sample4['price'])

MannwhitneyuResult(statistic=251.0, pvalue=0.45013346898056406)

## 2.2.3. Kolmogorov-Smirnov test
compares the underlying continuous distributions F(x) and G(x) of two independent samples.

In [141]:
ss.ks_2samp(sample2['price'],sample3['price'])

KstestResult(statistic=1.0, pvalue=3.715691111961505e-12)

In [146]:
ss.ks_2samp(sample['price'],sample4['price'])

KstestResult(statistic=0.4523809523809524, pvalue=0.02317953445823815)

## 2.2.4. Epps-Singleton test
Test the null hypothesis that two samples have the same underlying probability distribution.

In [149]:
ss.epps_singleton_2samp(sample1['muherrik_guc2'],sample3['muherrik_guc2'])

Epps_Singleton_2sampResult(statistic=1.0510906200754149, pvalue=0.901952533839152)

# 3. K sample tests

## 3.1. Parametric tests

## 3.1.1. Bartlett’s test
tests the null hypothesis that all input samples are from populations with equal variances.

In [160]:
ss.bartlett(sample1['price'],sample5['price'],sample['price'])

BartlettResult(statistic=9.830840250271814, pvalue=0.007332636620242348)

In [162]:
ss.bartlett(sample1['shekil_sayi'],sample5['shekil_sayi'],sample['shekil_sayi'])

BartlettResult(statistic=0.7417191002193877, pvalue=0.6901408649609686)

## 3.1.2. Levene test
tests the null hypothesis that all input samples are from populations with equal variances. 
more robust than Bartlett's test

In [161]:
ss.bartlett(sample1['price'],sample5['price'],sample['price'])

BartlettResult(statistic=9.830840250271814, pvalue=0.007332636620242348)

In [163]:
ss.bartlett(sample1['shekil_sayi'],sample5['shekil_sayi'],sample['shekil_sayi'])

BartlettResult(statistic=0.7417191002193877, pvalue=0.6901408649609686)

## 3.1.3. One-way ANOVA test
tests the null hypothesis that two or more groups have the same population mean. 

In [164]:
ss.f_oneway(sample['price'],sample1['price'],sample2['price'])

F_onewayResult(statistic=6.139026095885037, pvalue=0.003783881760406545)

In [166]:
ss.f_oneway(sample3['price'],sample1['price'],sample5['price'])

F_onewayResult(statistic=1.3607886129350237, pvalue=0.26425814859979774)

## 3.2. Non-parametric tests

## 3.2.1. Kruskal-Wallis H-test 
Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA.

In [172]:
ss.kruskal(sample3['price'],sample1['price'],sample5['price'])

KruskalResult(statistic=11.322042925950583, pvalue=0.003478961452326621)

In [175]:
ss.kruskal(sample2['price'],sample4['price'])

KruskalResult(statistic=0.589484706573294, pvalue=0.44261844868695854)

# 4. Correlation

## 4.1. Pearson r test - Parametric
Returns pearson correlation coefficient and p-value for testing non-correlation.

In [177]:
s1=df.sample(100)

In [180]:
ss.pearsonr(s1['price'],s1['shekil_sayi'])

(0.45743197481074316, 1.713399005185131e-06)

In [183]:
ss.pearsonr(s1['il'],s1['shekil_sayi'])

(0.12476271271774597, 0.21616869838534106)

## 4.2. Spearman $\rho$  test - Non-parametric
Non-parametric alternative of Pearson r test

In [179]:
ss.spearmanr(s1['marka'],s1['shekil_sayi'])

SpearmanrResult(correlation=0.08929633478180705, pvalue=0.3769643985121569)